In [1]:
pip install selenium

In [11]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

chromedriver.exe 경로 수정

In [12]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe'

In [13]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [14]:
query = '풀무원 바른먹거리'

In [21]:
def biz_khan(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 #추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    last_page =(int(page)-1)*10+1#추가된 부분@@@@@@@@@@@@@@@@@@@@@@@@@
    domain = '천지일보'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2041&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #url = url_domain + query + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('div',class_="article-head-title").text)
                head_list.append(head)
                tags = soup.select_one('#article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

biz = biz_khan(query,driver_path,3)

In [22]:
biz.to_csv('천지일보.csv',encoding='UTF8',index=None)

In [23]:
test = pd.read_csv('천지일보.csv',encoding='UTF8')

In [24]:
test

,언론사,제목,내용
0,천지일보,풀무원 초고압 제면기술로 뽑은 ‘평양냉면’ 출시,천지일보황해연 기자 풀무원이 초고압 제면기술이 적용된 ‘풀무원 평양냉면’을 출시했다...
1,천지일보,풀무원 온라인 모니터 요원 ‘’ 24기 모집,천지일보황해연 기자 풀무원식품이 바른먹거리를 사랑하는 풀무원 온라인 소비자 모니터 ...
2,천지일보,풀무원 코로나19시대 맞아 ‘시니어 바른먹거리 교육’ 실시,천지일보정인선 기자 풀무원이 코로나19 시대를 맞아 단체 교육 대신 11로 ‘시니어...
3,천지일보,풀무원푸드머스 ‘킨더슐레보육경영연구소’와 바른먹거리 공급 체결,천지일보정인선 기자 풀무원 계열 식자재 유통기업 ㈜풀무원푸드머스대표 윤희선가 국내 ...
4,천지일보,풀무원건강생활 ‘온열 테라피 안마의자’ 출시 가전 렌탈 사업 본격 시작,천지일보정인선 기자 풀무원건강생활이 바른 먹거리 건강기능식품을 넘어 건강한 생활을 ...
5,천지일보,풀무원 온라인 소비자 모니터 요원 ‘’ 23기 모집,천지일보정인선 기자 풀무원식품이 바른먹거리를 사랑하는 풀무원 온라인 소비자 모니터 ...
6,천지일보,풀무원녹즙 셀프 건강 관리 위한 건강즙 선물세트 기획전 진행,천지일보정인선 기자 ㈜풀무원녹즙대표 김기석이 바쁜 일상에서 간편케 건강을 챙길 수 ...
7,천지일보,풀무원다논 아이러브요거트 ‘하루한컵 사랑한컵’ 건강 챙기기 캠페인 진행,천지일보정인선 기자 요거트 전문 기업 풀무원다논대표 정희련의 온 가족 활력 요거트 ...
8,천지일보,키자니아 부산 개장 후 첫 신규시설 ‘바른먹거리 스쿨’ 오픈,천지일보최유라 기자 풀무원 ‘바름먹거리 스쿨’이 서울에 이어 부산에도 등장했다 글로...
9,천지일보,풀무원다논 ‘아이러브요거트’ 브랜드 리뉴얼 진행,천지일보정인선 기자 요거트 전문 기업 풀무원다논대표 정희련의 온 가족 활력 요거트 ...


In [ ]:
#article-view-content-div